<a href="https://colab.research.google.com/github/sgupta-7/Group_A_PH582_PROJECT/blob/main/CODE_TEAM_A_SUBHAM_GUPTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CODE FOR PROJECT GROUP A USING THREE APPROACHES

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dropout
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import mean_squared_error

# 1ST APPROACH( CONVENTIONAL SINGLE TS APPROACH)

In [ ]:
#vector = v = [x y z]
def model(v,t):
    x = v[0]
    y = v[1]
    z = v[2]
    alpha = 15.6
    beta = 28
    m0   = -1.143
    m1   = -0.714
    h = m1*x + 0.5*(m0-m1)*(abs(x+1)-abs(x-1))
    dxdt = alpha*(y - x - h)
    dydt = x - y + z
    dzdt = -beta*y 
    return np.array([dxdt, dydt, dzdt])

In [ ]:
step = 1
def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

In [ ]:
# DATA GENERATION
# initial condition
# v0 = initial condition, Change timestep range in 2nd line
v0 = np.array([0.7,0.1,0.2])
t = np.linspace(0,2000,num=2000)
v = odeint(model,v0,t)

In [ ]:
f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(t,v[:,0], linewidth=2)
plt.plot(t,v[:,1], linewidth=5)
plt.plot(t,v[:,2], linewidth=2)
plt.xlabel('time')
plt.ylabel('y(t)')
plt.show()

In [ ]:
a = v[:,0]
b = v[:,1]
c = v[:,2]
a = a.reshape(len(a), 1)
b = b.reshape(len(b), 1)
c = c.reshape(len(c), 1)

In [ ]:
N = 2000
Tp = 1600
train,test = v[0:Tp,:], v[Tp:N,:]

In [ ]:
trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)
trainX = trainX.reshape(trainX.shape[0], step, trainX.shape[2])
testX = testX.reshape(testX.shape[0], step, testX.shape[2])

## 1. LSTM

In [ ]:
modelLSTM = Sequential()
modelLSTM.add(LSTM( 50 , input_shape=( None , 3 )))
modelLSTM.add(Dense( 3 ))
modelLSTM.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
modelLSTM.summary()

In [ ]:
modelLSTM.fit(trainX,trainY,verbose= 2 ,epochs= 200, batch_size=32)

In [ ]:
trainPredict_LSTM = modelLSTM.predict(trainX)
testPredict_LSTM = modelLSTM.predict(testX)

In [ ]:
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict_LSTM))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict_LSTM))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# PLOTS
f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,0],color="red", linewidth=5,label="Original")
plt.plot(testPredict_LSTM[:,0],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('X',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,1],color="red", linewidth=5,label="Original")
plt.plot(testPredict_LSTM[:,1],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('Y',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,2],color="red", linewidth=5,label="Original")
plt.plot(testPredict_LSTM[:,2],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('Z',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

## 2. RNN

In [ ]:
RNNmodel = Sequential()
RNNmodel.add(SimpleRNN( 50 , input_shape=( None , 3 )))
RNNmodel.add(Dense( 3 ))
RNNmodel.compile(loss= 'mse' , optimizer= 'adam' )
RNNmodel.fit(trainX,trainY,verbose= 2 ,epochs= 200, batch_size=32)
RNNmodel.summary()

In [ ]:
trainPredict_RNN = RNNmodel.predict(trainX)
testPredict_RNN = RNNmodel.predict(testX)

In [ ]:
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict_RNN))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict_RNN))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# PLOTS
f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,0],color="red", linewidth=5,label="Original")
plt.plot(testPredict_RNN[:,0],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('x',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,1],color="red", linewidth=5,label="Original")
plt.plot(testPredict_RNN[:,1],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('y',fontsize=60)
plt.legend(loc='best',fontsize=60)


f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,2],color="red", linewidth=5,label="Original")
plt.plot(testPredict_RNN[:,2],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('Z',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

## 3. GRU

In [ ]:
modelGRU = Sequential()
modelGRU.add(GRU( 50 , input_shape=( None , 3 )))
modelGRU.add(Dense( 3 ))
modelGRU.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
modelGRU.summary()

In [ ]:
modelGRU.fit(trainX,trainY,verbose= 2 ,epochs= 150, batch_size=32)

In [ ]:
trainPredict_GRU = modelGRU.predict(trainX)
testPredict_GRU = modelGRU.predict(testX)
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict_GRU))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict_GRU))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# PLOT
f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,0],color="red", linewidth=5,label="Original")
plt.plot(testPredict_GRU[:,0],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('x',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,1],color="red", linewidth=5,label="Original")
plt.plot(testPredict_GRU[:,1],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('y',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(v[Tp+1:,2],color="red", linewidth=5,label="Original")
plt.plot(testPredict_GRU[:,2],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('z',fontsize=60)
plt.legend(loc='best',fontsize=60)
plt.show()

# 2ND APPROACH(  MULTI TS APPROACH)(SEQUENCE TO SEQUENCE)

In [ ]:
# n_past = This number refers to the timesteps in the input
# n_features = This number refers to the number of features
# n_future = This number refers to the timesteps in the output
n_past= 10
n_features = 3
n_future = 15

In [ ]:
def split_series(series, n_past, n_future):
    
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [ ]:
# Initial condition
# DATA GENERATION
v0 = np.array([0.7,0.1,0.2])
t = np.linspace(0,2000,num=2000)
v = odeint(model,v0,t)

In [ ]:
X_train, y_train = split_series(train,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)
#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)
model_e1d1.summary()

In [ ]:
model_e1d1.compile(loss= 'mean_squared_error' , optimizer= 'adam' )
model_e1d1.fit(X_train,y_train,verbose= 2 ,epochs= 100 )
testPredict_whole = model_e1d1.predict(X_test)
trainPredict_whole = model_e1d1.predict(X_train)

In [ ]:
trainScore = math.sqrt(mean_squared_error(y_train[:,0,:], trainPredict_whole[:,0,:]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0,:], testPredict_whole[:,0,:]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# PLOTS
f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(y_test[:,1,0],color="red", linewidth=5,label="Original")
plt.plot(testPredict_whole[:,1,0],color="black", linewidth=5,label="Predicted")
plt.xlabel('time',fontsize=60)
plt.ylabel('x',fontsize=60)
#plt.legend(loc='upper right', prop={'size': 60})
plt.legend(loc='upper right',fontsize=60)
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(y_test[:,1,1],color="red", linewidth=5,label="Original")
plt.plot(testPredict_whole[:,1,1],color="black", linewidth=5,label="Predicted")
plt.xlabel('time')
plt.ylabel('y')
plt.legend(loc='best')
plt.show()

f = plt.figure()
f.set_figwidth(70)
f.set_figheight(20)
plt.plot(y_test[:,1,2],color="red", linewidth=5,label="Original")
plt.plot(testPredict_whole[:,1,2],color="black", linewidth=5,label="Predicted")
plt.xlabel('time')
plt.ylabel('z')
plt.legend(loc='best')
plt.show()

# 3RD APPROACH(GENERATIVE MODE APPROACH) (LSTM)

In [ ]:
# CHANGE Tp FOR TRAIN RATIO
N = 20000
Tp = int(0.9*N)

In [ ]:
# Initial condition
# DATA GENERATION
v0 = np.array([0.7,0.1,0.2])
t = np.linspace(0,1000,num=N)
v = odeint(model,v0,t)

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
f = plt.figure()
f.set_figwidth(100)
f.set_figheight(50)
ax = plt.axes(projection='3d')
ax.plot3D(v[:,0], v[:,1], v[:,2], 'red')

In [ ]:
a = v[:,0]
b = v[:,1]
c = v[:,2]
a = a.reshape(len(a), 1)
b = b.reshape(len(b), 1)
c = c.reshape(len(c), 1)

In [ ]:
train,test = v[0:Tp,:], v[Tp:N,:]
trainX,trainY =convertToMatrix(train,step)
testX,testY =convertToMatrix(test,step)
trainX = trainX.reshape(trainX.shape[0], step, trainX.shape[2])
testX = testX.reshape(testX.shape[0], step, testX.shape[2])

In [ ]:
modelLSTM = Sequential()
modelLSTM.add(LSTM(50, input_shape=( None , 3 )))
modelLSTM.add(Dense( 3 ))
modelLSTM.compile(loss= 'mean_squared_error' , optimizer= 'adam',metrics=['accuracy'] )
modelLSTM.summary()

In [ ]:
modelLSTM.fit(trainX,trainY,verbose= 2 ,epochs= 50,batch_size=20 )

In [ ]:
a=testX[0,:,:]
predictions = list()
for i in range(N-Tp):
    # make one-step forecast
    a = a.reshape(1, 1, 3)
    trainPredict = modelLSTM.predict(a)
    predictions.append(trainPredict)
    a =  trainPredict

In [ ]:
finalArray = np.asarray(predictions, dtype = np.float32)

In [ ]:
# change ts based on how many timesteps the prediction is wanted
ts=200
# Plot results
f = plt.figure()
plt.plot(v[Tp:(Tp+ts),0],color="red", linewidth=2,label="Original")
plt.plot(finalArray[0:ts,0,0],color="black", linewidth=2,label="Predicted")
plt.xlabel('time')
plt.ylabel('x')
plt.legend(loc='best')
plt.show()

f = plt.figure()
plt.plot(v[Tp:(Tp+ts),1],color="red", linewidth=2,label="Original")
plt.plot(finalArray[0:ts,0,1],color="black", linewidth=2,label="Predicted")
plt.xlabel('time')
plt.ylabel('y')
plt.legend(loc='best')
plt.show()
f = plt.figure()

plt.plot(v[Tp:(Tp+ts),2],color="red", linewidth=2,label="Original")
plt.plot(finalArray[0:ts,0,2],color="black", linewidth=2,label="Predicted")
plt.xlabel('time')
plt.ylabel('z')
plt.legend(loc='best')
plt.show()
f = plt.figure()
f.set_figwidth(100)
f.set_figheight(10)

In [ ]:
# SCROLL PLOT
ts=N-Tp
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
f = plt.figure()
f.set_figwidth(80)
f.set_figheight(50)
ax = plt.axes(projection='3d')
ax.plot3D(v[Tp:(Tp+ts),0], v[Tp:(Tp+ts),1], v[Tp:(Tp+ts),2], 'red',linewidth=5)
plt.legend(loc='best')

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
f = plt.figure()
f.set_figwidth(80)
f.set_figheight(50)
ax = plt.axes(projection='3d')
ax.plot3D(finalArray[:ts,0,0], finalArray[:ts,0,1], finalArray[:ts,0,2], 'black',linewidth=5)
plt.legend(loc='best')